In [ ]:
# Note: 
# This project is to show what Neural Network training looks like under the hood.
# We will be building micrograd from andrej karpathy's micrograd project.

# Micrograd is a tiny autograd(automatic gradient) engine and neural network library written in Python.

# what it does is it implments backpropagation - backpropagation is the algorithm that allows you to efficiently compute the gradients of a loss function with respect to the weight of a neural network.
# This allows us to iteratively tune the weight of the neural network to minimize the loss function and improve the accuracy of the network

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [6]:
def f(x):
    return 3*x**2 - 4*x + 5

In [7]:
f(3.0)

20.0

In [ ]:
# to see the shape of the fuction f(x)   - f of x
